# Named Entity Recognition with Huggingface Trainer and 🔭 Galileo

In this tutorial, we'll train a model with Huggingface Trainer and explore the results in Galileo.

**Make sure to select GPU in your Runtime! (Runtime -> Change Runtime type)**

In [ ]:
#@title Install `dataquality`

try:
    import dataquality as dq
except ImportError:
    # Upgrade pip
    !pip install -U pip &> /dev/null

    # Install all dependecies
    !pip install -U dataquality seqeval evaluate torch datasets transformers &> /dev/null
    
    print('👋 Installed necessary libraries and restarting runtime! This should only need to happen once.')
    print('🙏 Continue with the rest of the notebook or hit "Run All" again!')

    # Restart the runtime
    import os, time
    time.sleep(1) # gives the print statements time to flush
    os._exit(0) # exits without allowing the next cell to run

# 1. Login to Galileo

In [ ]:
import dataquality as dq
dq.login()

# 2. Load Data

In [ ]:
#@title 🤗 HuggingFace Dataset
#@markdown You can select any dataset from [here](https://huggingface.co/datasets?language=language:en&task_categories=task_categories:token-classification&task_ids=task_ids:named-entity-recognition&sort=downloads) which contains train/validation/test splits and an `ner_tags` column.

dataset_name = 'conllpp' #@param ["wnut_17", "conllpp", "wikiann"] {allow-input: true}
print(f"You selected the {dataset_name} dataset")

from IPython.utils import io
from datasets import load_dataset, get_dataset_config_names

# Try to load the data. If a config (subset) is needed, pick one
try:
  with io.capture_output() as captured:
    dataset = load_dataset(dataset_name)
except ValueError as e:
  if "Config name is missing" not in repr(e):
    raise e

  configs = get_dataset_config_names(dataset_name)
  print(f"The dataset {dataset_name} has multiple subsets {configs}.")
  config = input(f"🖖 Enter the name of the subset to pick (or leave blank for any): ")
  if config:
    assert config in configs, f"{config} is not a valid subset"
  else:
    config = configs[0]
  with io.capture_output() as captured:
    dataset = load_dataset(dataset_name, name=config)

# Check that the dataset has at least train and validation splits
assert {"train", "validation", "test"}.issubset(dataset), \
f"💾 The dataset {dataset_name} does no have train/validation splits, please pick another one."

print(f"\n🏆 Dataset {dataset_name} loaded succesfully")

# A small function for minimizing the dataset for testing
import os

def _minimize_for_ci() -> bool:
    return os.getenv("MINIMIZE_FOR_CI", "false") == "true"

# 3. Initialize Galileo

In [ ]:
# 🔭🌕 Initializing a new run in Galileo. Each run goes under a project.
dq.init(task_type="text_ner", 
        project_name="named_entity_recognition_pytorch", 
        run_name=f"example_run_{dataset_name.replace('/','-')}_15")

# 4. Tokenize and Log Dataset
Galileo provides HF integrations to allow tokenization, and label alignment, while automatically logging your input data. Additionally, you can also use the `get_dataloader` function instead of PyTorch's native `DataLoader` to autoatically create a `Dataset` class, and prepare a dataset loader for each split.  

In [ ]:
from dataquality.integrations import hf
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('roberta-base', add_prefix_space=True)

# 🔭🌕 Galileo tokenizes the HuggingFace DatasetDict logs the dataset(s) present in it
tokenized_datasets = hf.tokenize_and_log_dataset(dataset, tokenizer)
labels = tokenized_datasets['train'].features['ner_tags'].feature.names  

if _minimize_for_ci():
  tokenized_datasets['train'] = tokenized_datasets['train'].select(range(10))
  tokenized_datasets['validation'] = tokenized_datasets['validation'].select(range(10))
  labels = tokenized_datasets['train'].features['ner_tags'].feature.names 

# 5. Prepare and Log NER Model 

One line of Galileo code to log the NER Model data (via `log_model_outputs`). This step logs the model logits and embeddings. 

In [ ]:
from transformers import (AutoModelForTokenClassification, DataCollatorForTokenClassification,
                          Trainer, TrainingArguments)
from datasets import load_dataset, load_metric

task = "ner"
label_list = dataset['train'].features['ner_tags'].feature.names
id2label = {idx:label for idx, label in enumerate(label_list)}
label2id = {label:idx for idx, label in id2label.items()}
label_all_token = True

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric('seqeval')
model = AutoModelForTokenClassification.from_pretrained('roberta-base', num_labels=len(label_list), id2label=id2label, label2id=label2id, finetuning_task=task)
batch_size = 10


def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


args = TrainingArguments(
    f"task-{task}",
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy='epoch',
    logging_strategy='epoch',
    logging_dir='./logs'
)


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 6. Putting into Action: Training a Model

Complete the training pipeline using a standard PyTorch training setup. While training, Galileo logs the current `epoch` and `split`. 

Call `dq.finish()` after training to complete the logging.

In [ ]:
from dataquality.integrations.transformers_trainer import watch

# 🔭🌕 Galileo Logging
watch(trainer)

trainer.train()

# 🔭🌕 Galileo Logging
dq.finish()

# General Help and Docs
- To get help with your task's requirements, call `dq.get_data_logger().doc()`
- To see more general data and model logging docs, run `dq.docs()`

In [ ]:
dq.get_data_logger().doc()
help(dq.log_dataset)